# Semantic Segmentation of Colon Polyps from Colonscopy Video: 
## UNet Model | CS230 | Christian H. Nunez

In [ ]:
# Imports
from __future__ import division
import os
import sys
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import json,codecs
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 14

## Seeding 
seed = 2019
random.seed = seed
#np.random.seed = seed
tf.seed = seed

print(keras.__version__)

In [ ]:
# Utilities for tuning
def saveHist(path,history):
    new_hist = {}
    for key in list(history.history.keys()):
        if type(history.history[key]) == np.ndarray:
            new_hist[key] == history.history[key].tolist()
        elif type(history.history[key]) == list:
           if  type(history.history[key][0]) == np.float64:
               new_hist[key] = list(map(float, history.history[key]))

    print(new_hist)
    with codecs.open(path, 'w', encoding='utf-8') as f:
        json.dump(new_hist, f, separators=(',', ':'), sort_keys=True, indent=4) 

def loadHist(path):
    with codecs.open(path, 'r', encoding='utf-8') as f:
        n = json.loads(f.read())
    return n

#Credit: https://stackoverflow.com/questions/41061457/keras-how-to-save-the-training-history/53101097

## Baseline Work (U-Net, no data augmentation)

In [ ]:
# Custom data generator (easy to use when analyzing predictions)
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
     
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, "Org", id_name) + ".png"
        mask_path = os.path.join(self.path, "GT", id_name) + ".png"
        
        ## Reading Image
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        #image = np.expand_dims(image, axis=-1)
        
        ## Reading Masks
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (self.image_size, self.image_size))
        mask = np.expand_dims(mask, axis=-1)
        #mask = np.maximum(np.zeros((self.image_size, self.image_size, 1)), mask)
                    
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

# Cell credit: https://github.com/nikhilroxtomar/UNet-Segmentation-in-Keras-TensorFlow/blob/master/unet-segmentation.ipynb
# __load__ function modified significantly to account for different file structure

In [ ]:
image_size = 128
train_path = "/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/"
epochs = 10
batch_size = 8

# Data Sizes
val_data_size = 18
test_data_size = 12

## Get training ids from original photo files
train_ids = os.listdir(train_path + "Org_9/")
total_images = len(train_ids)
for i in range(len(train_ids)):
    train_ids[i] = train_ids[i].split(".")[0]

# Choose train, train-dev (validation), dev, test split:
# 1 : Train and Train-Dev come from the same set of videos (SET A),
#     dev comes from a different set of videos (SET B), 
#     and test comes from a different set of videos (SET C).
#
# 0 : Random shuffle into train, train-dev, test.
option = 1
valid_ids = []
dev_ids = []
test_ids = []

if option == 1:
    train_ids = (np.sort(np.array(train_ids).astype(int))).astype(str)
    # Take 1.png through 25.png as the test set (all from one video)
    test_ids = train_ids[:25]
    # Take 26.png through 50.png as the dev set (all from one video)
    dev_ids = train_ids[25:50]
    # Split train_ids into train_ids and valid_ids
    train_ids = train_ids[50:]
    random.shuffle(train_ids)
    valid_ids = train_ids[:val_data_size]
    train_ids = train_ids[val_data_size:]
    
else:
    random.shuffle(train_ids)
    valid_ids = train_ids[:val_data_size]
    test_ids = train_ids[val_data_size:(val_data_size+test_data_size)]
    train_ids = train_ids[(val_data_size+test_data_size):]

print(" ==== TRAIN-DEV-TEST ==== ")
print("Image size %d x %d"%(image_size, image_size))
print("Total images: %d"%total_images)
print("Train data: %d frames (%s%%)"%(len(train_ids), (str)(len(train_ids) * 100/total_images)))
print("Validation data: %d frames (%s%%)"%(len(valid_ids), (str)(len(valid_ids)*100/total_images)))
print("Dev data: %d frames (%s%%)"%(len(dev_ids), (str)(len(dev_ids)*100/total_images)))
print("Test data: %d frames (%s%%)"%(len(test_ids), (str)(len(test_ids)*100/total_images)))

In [ ]:
gen = DataGen(train_ids, train_path, batch_size=len(train_ids), image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

In [ ]:
# View a random training image and mask:
r = random.randint(0, len(x)-1) + 4
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(x[r])
ax.set_title("Actual Image")
ax = fig.add_subplot(1, 2, 2)
ax.set_title("Ground Truth Mask")
ax.imshow(np.reshape(y[r], (image_size, image_size)), cmap="gray")
plt.savefig('examplepair.png')

In [ ]:
# Experimenting with overlaying the mask on the image (for presentation, not for the model)
def mask_overlay(image, mask, color=(0, 255, 0)):
    """
    Helper function to visualize mask on the top of the car
    """
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    mask = mask.astype(np.uint8)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0    
    img[ind] = weighted_sum[ind]    
    return img
# Cell credit: https://github.com/ternaus/robot-surgery-segmentation/blob/master/Demo.ipynb

In [ ]:
# Testing the overlay:
def load_image(path):
    img = cv2.imread(str(path))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

im = load_image("/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/Org/188.png")
msk = cv2.imread("/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/GT/188.png", 0)
msk = np.expand_dims(msk, axis=-1)
a = mask_overlay(im, (msk>0).astype(np.uint8))
img = Image.fromarray(a, 'RGB')


# Plot the figures:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(im)
ax[1].imshow(img)
ax[2].imshow(np.reshape(msk, (im.shape[0], im.shape[1])), cmap="gray")

# UNet Architecture Construction

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

# Cell credit: https://github.com/nikhilroxtomar/UNet-Segmentation-in-Keras-TensorFlow/blob/master/unet-segmentation.ipynb

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

# Cell credit: https://github.com/nikhilroxtomar/UNet-Segmentation-in-Keras-TensorFlow/blob/master/unet-segmentation.ipynb

### Define a Custom Loss (Dice Coefficient Loss)

In [ ]:
smooth = 1
def dice_coef(y_true, y_pred):
    y_true_f = keras.backend.flatten(y_true)
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# Dice coefficient implementation: https://forums.fast.ai/t/understanding-the-dice-coefficient/5838

In [ ]:
model = UNet()
adam = keras.optimizers.Adam(lr=.0001)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef, "binary_accuracy", 'mse'])
model.summary()

# Training Phase

In [ ]:
# Generate training and validation batches
train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

# Compute training and validation steps
train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

In [ ]:
# === Optional: Preload weights
#model.load_weights("/Users/ChristianHaroldNunez/Desktop/colons/best1.h5")
# === End preload weights

# Standard
epochs = 20
model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=epochs)

In [ ]:
# Save the weights
model.save_weights("20_lr0001.h5")

In [ ]:
# View an example of the prediction on one of the validation (train-dev) images
x, y = valid_gen.__getitem__(1)
print(valid_gen.__len__())
result = model.predict(x)
result = result > 0.5

fig, ax = plt.subplots(1, 3)

# Predicted Mask
ax[0].set_title("Actual Mask")
ax[0].imshow(np.reshape(y[0]*255, (image_size, image_size)), cmap="gray")

# Actual Mask'
ax[1].set_title("Predicted Mask")
ax[1].imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")

# Actual Image
ax[2].set_title("Actual Image")
ax[2].imshow(np.reshape(x[0][:,:,0]*255, (image_size, image_size)), cmap="gray")

# Predictions with the Dev Set

In [ ]:
# Test time
# ==============
# Create datagen for the dev and test sets:
test_gen = DataGen(test_ids, train_path, image_size=image_size, batch_size=len(test_ids))
dev_gen = DataGen(dev_ids, train_path, image_size=image_size, batch_size=len(dev_ids))

x, y = dev_gen.__getitem__(0)
print(x.shape, y.shape)

In [ ]:
# Predict with the images in the test:
result = model.predict(x)
result = result > 0.5

fig, ax = plt.subplots(1, 3)

# SAVE ALL TEST_IMAGE TRIPLETS FOR VIEWING
for n in range(len(test_ids)):
    ax[0].set_title("Actual Mask")
    ax[0].imshow(np.reshape(y[n]*255, (image_size, image_size)), cmap="gray")

    # Actual Mask'
    ax[1].set_title("Predicted Mask")
    ax[1].imshow(np.reshape(result[n]*255, (image_size, image_size)), cmap="gray")

    # Actual Image
    ax[2].set_title("Actual Image")
    ax[2].imshow(np.reshape(x[n][:,:,0]*255, (image_size, image_size)), cmap="gray")

    plt.savefig("/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/saved_test_figs/"+str(test_ids[n]))
    
plt.close()

# Example output included in milestone report.

In [ ]:
print(model.metrics_names)
model.evaluate(x, y, batch_size=len(x), verbose=1)

# Improved U-Net: Data Augmentation and Tuning

In [ ]:
model = UNet()
adam = keras.optimizers.Adam(lr=.0001)
rmsprop = keras.optimizers.RMSprop(lr=.0001)
sgd = keras.optimizers.SGD(lr=.001, momentum=0.9)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef, "binary_accuracy", 'mse'])
#model.load_weights("20_lr0001.h5")
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import itertools as itt


def combineGenerator(gen1, gen2):
    while True:
        #print(gen1.next()[0].shape, gen2.next()[0].shape)
        yield(gen1.next()[0], gen2.next()[0])

epochs = 20
batch_size = 1
data_aug_path = '/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/data_aug2/'

# Augmentation configuration for training:
#--------
# data_gen_args=dict(rescale=1./255,
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.1,
#         brightness_range=(0.1, 0.8),
#         horizontal_flip=True)

data_gen_args=dict(rescale=1./255,
        rotation_range=360,    
        brightness_range=(0.4, 0.6),
        vertical_flip=True,
        horizontal_flip=True)


#data_gen_args=dict(rescale=1./255)
# -------

# Making image data for TRAINING images
train_image_datagen = ImageDataGenerator(**data_gen_args)
train_mask_datagen = ImageDataGenerator(**data_gen_args)

seed1 = 230

train_image_generator = train_image_datagen.flow_from_directory(data_aug_path+'train/Org_upper',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=seed1)

train_mask_generator = train_mask_datagen.flow_from_directory(data_aug_path+'train/GT_upper',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=seed1,
    color_mode='grayscale')

train_generator = combineGenerator(train_image_generator, train_mask_generator)

# Making image data for VALIDATION images
validation_image_datagen = ImageDataGenerator(rescale=1./255)
validation_mask_datagen = ImageDataGenerator(rescale=1./255)

validation_image_generator = validation_image_datagen.flow_from_directory(data_aug_path+'val/Org_upper',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=seed1)

validation_mask_generator = validation_mask_datagen.flow_from_directory(data_aug_path+'val/GT_upper',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed=seed1,
    color_mode='grayscale')

validation_generator = combineGenerator(validation_image_generator, validation_mask_generator)


# Compute training and validation steps
train_steps = 425//batch_size
valid_steps = 50//batch_size

print(type(validation_generator))

history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_steps,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=valid_steps
)

model.save_weights('data_aug_HP6_nopretrained_lr0001_batch50.h5')

In [ ]:
plt.plot(history.history['dice_coef'], marker='o')
plt.plot(history.history['val_dice_coef'], marker='o')

In [ ]:
plt.plot(history.history['loss'], marker='o')
plt.plot(history.history['val_loss'], marker='o')

In [ ]:
# SAVE YOUR HISTORY:
saveHist('/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/HP5/data_aug_HP6_nopretrained_lr0001_batch50.json', history)

In [ ]:
history.history.keys()

In [ ]:
# Test time
# ==============
# Create datagen for the dev and test sets:
test_gen = DataGen(test_ids, train_path, image_size=image_size, batch_size=len(test_ids))
dev_gen = DataGen(dev_ids, train_path, image_size=image_size, batch_size=len(dev_ids))

x, y = test_gen.__getitem__(0)
print(x.shape, y.shape)

print(model.metrics_names)
model.evaluate(x, y, batch_size=len(x), verbose=1)

In [ ]:
# Get the maximum training dice:
print("Max train dice_coef:", max(history.history["dice_coef"]))
print("Max val_dice_coef: ", max(history.history["val_dice_coef"]))

In [ ]:
# ---------------------------
# PREPARING TRAINING/DEV SETS
# ---------------------------
image_size1 = 128
train_path1 = "/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/"

# Training set image demarcations


## Get training ids from original photo files
train_ids1 = os.listdir(train_path1 + "Org/")
total_images1 = len(train_ids1)
for i in range(len(train_ids1)):
    train_ids1[i] = train_ids1[i].split(".")[0]
    
# Initialize ID lists
valid_ids1 = []
dev_ids1 = []
test_ids1 = []

train_ids1 = (np.sort(np.array(train_ids1).astype(int))).astype(str)

seq_ends = [0, 25, 50, 67, 78, 103, 126, 151, 177]

evals = []
for s in range(2, len(seq_ends)-1):
    tester_ids = train_ids1[seq_ends[s]:seq_ends[s+1]]
    print(tester_ids)
    # Test of those^
    test_genner = DataGen(tester_ids, train_path1, image_size=image_size1, batch_size=len(tester_ids))
    x, y = test_genner.__getitem__(0)
    evals.append(model.evaluate(x, y, batch_size=len(x), verbose=1))

In [ ]:
tot = 0
for i in range(len(evals)):
    tot = tot + evals[i][1]
    print("dice_coef: ", evals[i][1])

print("avg in test set: ", tot/len(evals))

In [ ]:
# Predict with the images in the test:
result = model.predict(x)
result = result > 0.5

fig, ax = plt.subplots(1, 3)

# SAVE ALL TEST_IMAGE TRIPLETS FOR VIEWING
for n in range(len(test_ids)):
    ax[0].set_title("Actual Mask")
    ax[0].imshow(np.reshape(y[n]*255, (image_size, image_size)), cmap="gray")

    # Actual Mask'
    ax[1].set_title("Predicted Mask")
    ax[1].imshow(np.reshape(result[n]*255, (image_size, image_size)), cmap="gray")

    # Actual Image
    ax[2].set_title("Actual Image")
    ax[2].imshow(np.reshape(x[n][:,:,0]*255, (image_size, image_size)), cmap="gray")

    plt.savefig("/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/HP2/saved_test_figs_TEST_Seq1_60+180/"+str(test_ids[n]))
    
plt.close()

In [ ]:
# DATA AUGMENTATION EXPERIMENTS:
# Use this to output preview augmented images to a folder.

data_gen_args=dict(rescale=1./255,
        zca_whitening=True,
        rotation_range=360,    
        brightness_range=(0.1, 0.9),
        vertical_flip=True,
        horizontal_flip=True)

datagen = ImageDataGenerator(**data_gen_args)

num = '512'
img = load_img("/Users/ChristianHaroldNunez/Desktop/colons/CVC-ClinicDB/GT/" + num + ".png")  # this is a PIL image
x = img_to_array(img)

print(type(x), x.shape)
x = x.reshape((1,) + x.shape)

# Saving some example augmented images to my "previews" directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='/Users/ChristianHaroldNunez/Desktop/colons/previews', save_prefix=num, save_format='png'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely
        
# Inspiration: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html